# scRFE main code 

## Setup 

In [1]:
# Imports 
import numpy as np
import pandas as pd
import scanpy as sc
from anndata import read_h5ad
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

Read in your anndata object, which will become 'tiss'

In [2]:
adata = read_h5ad('/Users/madelinepark/Downloads/Limb_Muscle_facs.h5ad')
tiss = adata

## Random Forest 

In [3]:
def scRFE (tiss, feature='age', n_estimators=1000, random_state=0, n_jobs=-1, oob_score=True, test_size = 0.05, step=0.2, cv=5) :
    """ 
    Gives list of genes and corresponding ginis for given feature of interest 
   
    Enter your input data (anndata object) and select feature of interest.
    Loop through each type of the given feature and find genes most important in 
    describing that feature. 
    
    :param input_set: anndata object
        .h5ad file from figshare 
    :param feature: str 
        pick any column from tiss.obs (ex: age, cell_ontology_class)
    :param n_estimators : integer, optional (default=1000)
        the number of trees in the forest.
        see sklearn.ensemble.RandomForestClassifier for parameter details.
    :random_state : int, (default=0)
        see sklearn.model_selection.train_test_split for parameter details.
    :param n_jobs : int or None, (default=-1)
        The number of jobs to run in parallel for both fit and predict. 
        -1 means using all processors. 
        see sklearn.ensemble.RandomForestClassifier for parameter details.        
    :param oob_score : bool (default=True)
        Uses out-of-bag samples to estimate the generalization accuracy.
        see sklearn.ensemble.RandomForestClassifier for parameter details.
    :param step: int or float, optional (default=0.2)
        Step corresponds to the percentage of features to remove at each iteration.
        see sklearn.feature_selection.RFECV for parameter details.
    :param cv: int, cross-validation generator or an iterable, optional (default = 5)
        Determines the cross-validation splitting strategy. 
        Greater cv more accurate, takes longer to run.
        see sklearn.feature_selection.RFECV for parameter details.
    :param test_size: float, int or None (default = 0.05)
        Represents the proportion of the dataset to include in the test split.
        see sklearn.model_selection.train_test_split for parameter details.
    """
    
    tiss.obs['feature_type_of_interest'] = 'rest'
    results_feature_cv = pd.DataFrame()
    for c in list(set(tiss.obs[feature])): 
        print(c)
        clf = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=-1, oob_score=True)
        selector = RFECV(clf, step=0.2, cv=3, n_jobs=4) # step = % rounded down at each iteration  
        feature_of_interest = c

        tiss.obs.loc[tiss.obs[tiss.obs[feature] == feature_of_interest].index,'feature_type_of_interest'] = feature_of_interest

        feat_labels = tiss.var_names 
        X = tiss.X
        y = tiss.obs['feature_type_of_interest']

        print('training...')
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0) 

        clf.fit(X_train, y_train)
        selector.fit(X_train, y_train)
        feature_selected = feat_labels[selector.support_] 

        print('result writing')

        column_headings = []
        column_headings.append(c)
        column_headings.append(c + '_gini')

        resaux = pd.DataFrame(columns=column_headings)
        resaux[c] = feature_selected
        resaux[c + '_gini'] = (selector.estimator_.feature_importances_)

        print(feature_selected)
        print (selector.estimator_.feature_importances_)

        results_feature_cv = pd.concat([results_feature_cv,resaux],axis=1)

        tiss.obs['feature_type_of_interest'] = 'rest'

    results_feature_cv

## Run the argument 

Be sure to specify your tiss anndata object, what feature you're interested in, how many estimators you want, step size, and cv.

In [5]:
scRFE(tiss=tiss, feature='age', n_estimators=10, step = 0.2, cv=2)

3m
training...


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

result writing
Index(['0610007P08Rik', '0610007P14Rik', '0610007P22Rik', '0610008F07Rik',
       '0610009B14Rik', '0610009B22Rik', '0610009D07Rik', '0610009L18Rik',
       '0610009O20Rik', '0610010B08Rik',
       ...
       'Zfp101', 'Zfp187', 'Zfp189', 'Zfp207', 'Zfp212', 'Zfp281', 'Zfp874a',
       'Zmynd8', 'Zranb1', 'zsGreen_transgene'],
      dtype='object', name='index', length=4583)
[0. 0. 0. ... 0. 0. 0.]
24m
training...


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

result writing
Index(['0610007P08Rik', '0610007P14Rik', '0610007P22Rik', '0610008F07Rik',
       '0610009B14Rik', '0610009B22Rik', '0610009D07Rik', '0610009L18Rik',
       '0610009O20Rik', '0610010B08Rik',
       ...
       'Zfp101', 'Zfp187', 'Zfp189', 'Zfp207', 'Zfp212', 'Zfp281', 'Zfp874a',
       'Zmynd8', 'Zranb1', 'zsGreen_transgene'],
      dtype='object', name='index', length=4583)
[0. 0. 0. ... 0. 0. 0.]
